### Notebook to calculate various volume-integrated quantities over the short domain for the 2X across-shore shelf 

In [1]:
#Packages 
import numpy as np
import xgcm
from xgcm import Grid
import xarray as xr
import xroms
from datetime import datetime

import glob
from xhistogram.xarray import histogram
import cmocean.cm as cmo
import matplotlib.pyplot as plt
from matplotlib import colors
import matplotlib.ticker as tick
from matplotlib.dates import DateFormatter
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.dates as mdates
from matplotlib.ticker import AutoMinorLocator
from xhistogram.xarray import histogram
from datetime import timedelta
import time

import warnings
warnings.filterwarnings("ignore") #The chaotic option, used to suppress issues with cf_time with xroms 

from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)
from matplotlib import pyplot as plt, patches

In [2]:
def open_roms(path):
    ds1 = xroms.open_netcdf(path)
    ds1, grid1 = xroms.roms_dataset(ds1)
    return ds1, grid1

# HSIMT
# paths = ['/d2/home/dylan/idealized_nummix/shelf_hsimt_uwind_zerop1_dt_30_across2x_50d_avg.nc',
#          '/d1/shared/shelfstrat_wind/tadv_ensembles/shelf_hsimt_uwind_zerop1_dt_30_across2x_50d_ensmb1_avg.nc',
#          '/d1/shared/shelfstrat_wind/tadv_ensembles/shelf_hsimt_uwind_zerop1_dt_30_across2x_50d_ensmb2_avg.nc',
#          '/d1/shared/shelfstrat_wind/tadv_ensembles/shelf_hsimt_uwind_zerop1_dt_30_across2x_50d_ensmb3_avg.nc',
#          '/d1/shared/shelfstrat_wind/tadv_ensembles/shelf_hsimt_uwind_zerop1_dt_30_across2x_50d_ensmb4_avg.nc',
#          '/d1/shared/shelfstrat_wind/tadv_ensembles/shelf_hsimt_uwind_zerop1_dt_30_across2x_50d_ensmb5_avg.nc',
#          '/d1/shared/shelfstrat_wind/tadv_ensembles/shelf_hsimt_uwind_zerop1_dt_30_across2x_50d_ensmb6_avg.nc',
#          '/d1/shared/shelfstrat_wind/tadv_ensembles/shelf_hsimt_uwind_zerop1_dt_30_across2x_50d_ensmb7_avg.nc',
#         ]
# MPDATA
# paths = ['/d2/home/dylan/idealized_nummix/shelf_mpdata_uwind_zerop1_dt_30_across2x_50d_avg.nc',
#          '/d1/shared/shelfstrat_wind/tadv_ensembles/shelf_mpdata_uwind_zerop1_dt_30_across2x_50d_ensmb1_avg.nc',
#          '/d1/shared/shelfstrat_wind/tadv_ensembles/shelf_mpdata_uwind_zerop1_dt_30_across2x_50d_ensmb2_avg.nc',
#          '/d1/shared/shelfstrat_wind/tadv_ensembles/shelf_mpdata_uwind_zerop1_dt_30_across2x_50d_ensmb3_avg.nc',
#          '/d1/shared/shelfstrat_wind/tadv_ensembles/shelf_mpdata_uwind_zerop1_dt_30_across2x_50d_ensmb4_avg.nc',
#          '/d1/shared/shelfstrat_wind/tadv_ensembles/shelf_mpdata_uwind_zerop1_dt_30_across2x_50d_ensmb5_avg.nc',
#          '/d1/shared/shelfstrat_wind/tadv_ensembles/shelf_mpdata_uwind_zerop1_dt_30_across2x_50d_ensmb6_avg.nc',
#          '/d1/shared/shelfstrat_wind/tadv_ensembles/shelf_mpdata_uwind_zerop1_dt_30_across2x_50d_ensmb7_avg.nc',
#         ]
# U3HC4
paths = ['/d2/home/dylan/idealized_nummix/shelf_u3hc4_uwind_zerop1_dt_30_across2x_50d_avg.nc',
         '/d1/shared/shelfstrat_wind/tadv_ensembles/shelf_u3hc4_uwind_zerop1_dt_30_across2x_50d_ensmb1_avg.nc',
         '/d1/shared/shelfstrat_wind/tadv_ensembles/shelf_u3hc4_uwind_zerop1_dt_30_across2x_50d_ensmb2_avg.nc',
         '/d1/shared/shelfstrat_wind/tadv_ensembles/shelf_u3hc4_uwind_zerop1_dt_30_across2x_50d_ensmb3_avg.nc',
         '/d1/shared/shelfstrat_wind/tadv_ensembles/shelf_u3hc4_uwind_zerop1_dt_30_across2x_50d_ensmb4_avg.nc',
         '/d1/shared/shelfstrat_wind/tadv_ensembles/shelf_u3hc4_uwind_zerop1_dt_30_across2x_50d_ensmb5_avg.nc',
         '/d1/shared/shelfstrat_wind/tadv_ensembles/shelf_u3hc4_uwind_zerop1_dt_30_across2x_50d_ensmb6_avg.nc',
         '/d1/shared/shelfstrat_wind/tadv_ensembles/shelf_u3hc4_uwind_zerop1_dt_30_across2x_50d_ensmb7_avg.nc',
        ]

def open_roms(path):
    ds1 = xroms.open_netcdf(path)
    ds1, grid1 = xroms.roms_dataset(ds1)
    ds1 = ds1.isel(ocean_time = slice(0,723))
    return ds1, grid1

ds = []
grid = []
for i in range(len(paths)):
    ds1, grid1 = open_roms(paths[i])
    ds.append(ds1)
    grid.append(grid1)
    # print('iter complete')

In [3]:
t = -1
for i in range(len(paths)):
    print(ds[i].ocean_time[t].values)
    print(len(ds[i].ocean_time))

0001-01-31 02:30:00
723
0001-01-30 23:30:00
720
0001-01-30 23:30:00
720
0001-01-30 23:30:00
720
0001-01-30 23:30:00
720
0001-01-30 23:30:00
720
0001-01-30 23:30:00
720
0001-01-30 23:30:00
720


In [4]:
def mixing_vint(ds,grid,xislice,etaslice):
    
    mnum = (ds.dye_03*ds.dV).isel(eta_rho = etaslice, xi_rho = xislice).sum(['s_rho','eta_rho','xi_rho'])
    mnum.attrs = ''
    mnum.name = 'mnum'
    
    Akr = ds.AKr
    AKr_rho = grid.interp(Akr, 'Z', boundary = 'extend')
    mphy = (AKr_rho*ds.dV).isel(eta_rho = etaslice, xi_rho = xislice).sum(['s_rho','eta_rho','xi_rho'])
    mphy.attrs = ''
    mphy.name = 'mphy'
    
    mt = ds.dye_03+AKr_rho
    mtot = (mt*ds.dV).isel(eta_rho = etaslice, xi_rho = xislice).sum(['s_rho','eta_rho','xi_rho'])
    mtot.attrs = ''
    mtot.name = 'mtot'
    
    ds_mix = xr.merge([mnum, mphy, mtot])
    return ds_mix

def mixing_vint_top1m(ds,grid,xislice,etaslice):
    
    mnum = (ds.dye_03*ds.dV).where(ds.z_rho>-1).isel(eta_rho = etaslice, xi_rho = xislice).sum(['s_rho','eta_rho','xi_rho'])
    mnum.attrs = ''
    mnum.name = 'mnum'
    
    Akr = ds.AKr
    AKr_rho = grid.interp(Akr, 'Z', boundary = 'extend')
    mphy = (AKr_rho*ds.dV).where(ds.z_rho>-1).isel(eta_rho = etaslice, xi_rho = xislice).sum(['s_rho','eta_rho','xi_rho'])
    mphy.attrs = ''
    mphy.name = 'mphy'
    
    mt = ds.dye_03+AKr_rho
    mtot = (mt*ds.dV).where(ds.z_rho>-1).isel(eta_rho = etaslice, xi_rho = xislice).sum(['s_rho','eta_rho','xi_rho'])
    mtot.attrs = ''
    mtot.name = 'mtot'
    
    ds_mix_1m = xr.merge([mnum, mphy, mtot])
    return ds_mix_1m

def energy_vint(ds,grid,xislice,etaslice):
    '''
Calculates volume-integrated eddy, mean, and total kinetic energy based on 
Hetland (2017) JPO.
Notes:
------
EKE = 1/2(uprime^2 + v^2). 
MKE = 1/2(ubar^2)
TKE = 1/2(u^2+v^2)
u = ubar+uprime, ubar = 1/L int_0^L u dx, i.e., alongshore mean
vbar = 0 by design.
Velocities interpolated to their respective rho points
    '''
    u = xroms.to_rho(ds.u, grid)
    u = u.isel(eta_rho = etaslice, xi_rho = xislice) 
    v = xroms.to_rho(ds.v, grid)
    v = v.isel(eta_rho = etaslice, xi_rho = xislice)
    ubar = u.mean('xi_rho')
    uprime = u-ubar
    vprime = v
    
    #Eddy kinetic energy
    eke = 0.5*(uprime**2 + vprime**2)
    dV = ds.dV.isel(eta_rho = etaslice, xi_rho = xislice)
    eke_int = (eke*dV).sum(['eta_rho', 'xi_rho', 's_rho'])
    eke_int.attrs = ''
    eke_int.name = 'eke'
    
    #Mean kinetic energy
    mke = 0.5*ubar**2
    mke_int = (mke*dV).sum(['eta_rho', 'xi_rho', 's_rho'])
    mke_int.attrs = ''
    mke_int.name = 'mke'
    
    #Total kinetic energy 
    tke = 0.5*(u**2+v**2)
    tke_int = (tke*dV).sum(['eta_rho', 'xi_rho', 's_rho'])
    tke_int.attrs = ''
    tke_int.name = 'tke'
    
    ds_energy = xr.merge([eke_int, mke_int, tke_int])
    return ds_energy

def sprime2_whole(ds, grid, xislice, etaslice):
    ''' 
Returns volume-averaged salinity variance. 
Inputs:
----
ds: DataArray
salt: DataArray
Outputs:
----
svar: total salinity variance
    '''
    salt = ds.salt.isel(eta_rho = etaslice, xi_rho = xislice)
    dV = ds.dV.isel(eta_rho = etaslice, xi_rho = xislice)
    V = dV.sum(['s_rho', 'xi_rho', 'eta_rho'])
    
    sbar = (1/V)*((salt*dV).sum(['s_rho', 'xi_rho', 'eta_rho']))
    sp_tot = (1/V)*((((salt-sbar)**2)*dV).sum(['s_rho', 'xi_rho', 'eta_rho']))
    
    return sp_tot


def sprime2_top1m(ds, grid, xislice, etaslice):
    ''' 
Returns volume-averaged salinity variance. 
Inputs:
----
ds: DataArray
salt: DataArray
Outputs:
----
svar: total salinity variance
    '''
    salt = ds.salt.where(ds.z_rho>-1).isel(eta_rho = etaslice, xi_rho = xislice)
    dV = ds.dV.where(ds.z_rho>-1).isel(eta_rho = etaslice, xi_rho = xislice)
    V = dV.sum(['s_rho', 'xi_rho', 'eta_rho'])
    
    sbar = (1/V)*((salt*dV).sum(['s_rho', 'xi_rho', 'eta_rho']))
    sp_1m = (1/V)*((((salt-sbar)**2)*dV).sum(['s_rho', 'xi_rho', 'eta_rho']))
    
    return sp_1m

In [5]:
# Run the functions
# -----------------
xislice = slice(1,-1)
etaslice = slice(1,193) 

# dsm = []
# for i in range(len(paths)):
#     ds_mix = mixing_vint(ds[i],grid[i],xislice,etaslice)
#     dsm.append(ds_mix)
    
# dsm_1m = []
# for i in range(len(paths)):
#     ds_mix_1m = mixing_vint_top1m(ds[i],grid[i],xislice,etaslice)
#     dsm_1m.append(ds_mix_1m)
    
# dse = []
# for i in range(len(paths)):
#     ds_energy = energy_vint(ds[i],grid[i],xislice,etaslice)
#     dse.append(ds_energy)
    
# svar = []
# for i in range(len(paths)):
#     sp_tot = sprime2_whole(ds[i], grid[i], xislice, etaslice)
#     svar.append(sp_tot)
    
# svar_1m = []
# for i in range(len(paths)):
#     sp_1m = sprime2_top1m(ds[i], grid[i], xislice, etaslice)
#     svar_1m.append(sp_1m)

In [6]:
def sprime2_whole(ds, grid, xislice, etaslice):
    ''' 
Returns the volume-averaged terms for the decomposition of salinity variance: 
total, vertical, and horizontal variance. See Li et al. (2018) JPO for details.
Inputs:
----
ds: DataArray
salt: DataArray
Outputs:
----
svar: total salinity variance
svert: vertical salinity variance
shorz: horizontal salinity variance
    '''
    salt = ds.salt.isel(eta_rho = etaslice, xi_rho = xislice)
    dV = ds.dV.isel(eta_rho = etaslice, xi_rho = xislice)
    V = dV.sum(['s_rho', 'xi_rho', 'eta_rho'])
    
    sbar = (1/V)*((salt*dV).sum(['s_rho', 'xi_rho', 'eta_rho']))
    sp_tot = (1/V)*((((salt-sbar)**2)*dV).sum(['s_rho', 'xi_rho', 'eta_rho']))
    
    # Now compute the local vertical salinity variance
    dz = ds.dz.isel(eta_rho = etaslice, xi_rho = xislice)
    Z = dz.sum(['s_rho'])
    sbar_z = (1/Z)*((salt*dz).sum(['s_rho']))
    sp_vert = (1/V)*((((salt-sbar_z)**2)*dV).sum(['s_rho', 'xi_rho', 'eta_rho']))
    
    sp_horz = (sp_tot-sp_vert)
    
    sp_tot.attrs = ''
    sp_tot.name = 'svar_tot'
    sp_vert.attrs = ''
    sp_vert.name = 'svar_vert'
    sp_horz.attrs = ''
    sp_horz.name = 'svar_horz'
    
    svar_da = xr.merge([sp_tot, sp_vert, sp_horz])
    
    return svar_da

svar = []
for i in range(len(paths)):
    svar_da = sprime2_whole(ds[i], grid[i], xislice, etaslice)
    svar.append(svar_da)
    
# svar = []
# for i in range(len(paths)):
#     sp_tot = sprime2_whole(ds[i], grid[i], xislice, etaslice)
#     svar.append(sp_tot)

### Run the functions starting from the nth ensemble compared to a complete list of paths (e.g., 0-n)
This can be easily adjusted if you want to rerun the calcs 

In [7]:
n = 0 # Start at th
# n = 5 # Start at the fifth ensemble
# MPDATA
# for i in range(len(paths)):
#     # svar[i].attrs = ''
#     # svar[i].name = 'svar'
#     p = 'svar_vavg_decomp_mpdata_'+'ensemble_'+str(i+n)+'.nc'
#     svar[i].to_netcdf(p, mode = 'w')
    
# for i in range(len(paths)):
#     svar_1m[i].attrs = ''
#     svar_1m[i].name = 'svar'
#     p = 'svar_vavg_top1m_mpdata_'+'ensemble_'+str(i+n)+'.nc'
#     svar_1m[i].to_netcdf(p, mode = 'w')

# for i in range(len(paths)):
#     p = 'ene_vint_whole_mpdata_'+'ensemble_'+str(i+n)+'.nc'
#     dse[i].to_netcdf(p, mode = 'w')
    
# for i in range(len(paths)):
#     p = 'mix_vint_whole_mpdata_'+'ensemble_'+str(i+n)+'.nc'
#     dsm[i].to_netcdf(p, mode = 'w')
    
# for i in range(len(paths)):
#     p = 'mix_vint_top1m_mpdata_'+'ensemble_'+str(i+n)+'.nc'
#     dsm_1m[i].to_netcdf(p, mode = 'w')

# HSIMT
# for i in range(len(paths)):
#     # svar[i].attrs = ''
#     # svar[i].name = 'svar'
#     p = 'svar_vavg_whole_hsimt_'+'ensemble_'+str(i+n)+'.nc'
#     svar[i].to_netcdf(p, mode = 'w')
    
# for i in range(len(paths)):
#     svar_1m[i].attrs = ''
#     svar_1m[i].name = 'svar'
#     p = 'svar_vavg_top1m_hsimt_'+'ensemble_'+str(i+n)+'.nc'
#     svar_1m[i].to_netcdf(p, mode = 'w')

# for i in range(len(paths)):
#     p = 'ene_vint_whole_hsimt_'+'ensemble_'+str(i+n)+'.nc'
#     dse[i].to_netcdf(p, mode = 'w')
    
# for i in range(len(paths)):
#     p = 'mix_vint_whole_hsimt_'+'ensemble_'+str(i+n)+'.nc'
#     dsm[i].to_netcdf(p, mode = 'w')
    
# for i in range(len(paths)):
#     p = 'mix_vint_top1m_hsimt_'+'ensemble_'+str(i+n)+'.nc'
#     dsm_1m[i].to_netcdf(p, mode = 'w')

# # U3HC4
for i in range(len(paths)):
    # svar[i].attrs = ''
    # svar[i].name = 'svar'
    p = 'svar_vavg_whole_u3hc4_'+'ensemble_'+str(i+n)+'.nc'
    svar[i].to_netcdf(p, mode = 'w')
    
# for i in range(len(paths)):
#     svar_1m[i].attrs = ''
#     svar_1m[i].name = 'svar'
#     p = 'svar_vavg_top1m_u3hc4_'+'ensemble_'+str(i+n)+'.nc'
#     svar_1m[i].to_netcdf(p, mode = 'w')

# for i in range(len(paths)):
#     p = 'ene_vint_whole_u3hc4_'+'ensemble_'+str(i+n)+'.nc'
#     dse[i].to_netcdf(p, mode = 'w')
    
# for i in range(len(paths)):
#     p = 'mix_vint_whole_u3hc4_'+'ensemble_'+str(i+n)+'.nc'
#     dsm[i].to_netcdf(p, mode = 'w')
    
# for i in range(len(paths)):
#     p = 'mix_vint_top1m_u3hc4_'+'ensemble_'+str(i+n)+'.nc'
#     dsm_1m[i].to_netcdf(p, mode = 'w') 

### Code for the entire path list at once
It is commented out as backup 

In [ ]:
# # HSIMT
# for i in range(len(paths)):
#     svar[i].attrs = ''
#     svar[i].name = 'svar'
#     p = 'svar_vavg_whole_hsimt_'+'ensemble_'+str(i)+'.nc'
#     svar[i].to_netcdf(p, mode = 'w')
    
# for i in range(len(paths)):
#     svar_1m[i].attrs = ''
#     svar_1m[i].name = 'svar'
#     p = 'svar_vavg_top1m_hsimt_'+'ensemble_'+str(i)+'.nc'
#     svar_1m[i].to_netcdf(p, mode = 'w')

# for i in range(len(paths)):
#     p = 'ene_vint_whole_hsimt_'+'ensemble_'+str(i)+'.nc'
#     dse[i].to_netcdf(p, mode = 'w')
    
# for i in range(len(paths)):
#     p = 'mix_vint_whole_hsimt_'+'ensemble_'+str(i)+'.nc'
#     dsm[i].to_netcdf(p, mode = 'w')
    
# for i in range(len(paths)):
#     p = 'mix_vint_top1m_hsimt_'+'ensemble_'+str(i)+'.nc'
#     dsm_1m[i].to_netcdf(p, mode = 'w')

# # MPDATA
# for i in range(len(paths)):
#     svar[i].attrs = ''
#     svar[i].name = 'svar'
#     p = 'svar_vavg_whole_mpdata_'+'ensemble_'+str(i)+'.nc'
#     svar[i].to_netcdf(p, mode = 'w')
    
# for i in range(len(paths)):
#     svar_1m[i].attrs = ''
#     svar_1m[i].name = 'svar'
#     p = 'svar_vavg_top1m_mpdata_'+'ensemble_'+str(i)+'.nc'
#     svar_1m[i].to_netcdf(p, mode = 'w')

# for i in range(len(paths)):
#     p = 'ene_vint_whole_mpdata_'+'ensemble_'+str(i)+'.nc'
#     dse[i].to_netcdf(p, mode = 'w')
    
# for i in range(len(paths)):
#     p = 'mix_vint_whole_mpdata_'+'ensemble_'+str(i)+'.nc'
#     dsm[i].to_netcdf(p, mode = 'w')
    
# for i in range(len(paths)):
#     p = 'mix_vint_top1m_mpdata_'+'ensemble_'+str(i)+'.nc'
#     dsm_1m[i].to_netcdf(p, mode = 'w')

# # U3HC4
# for i in range(len(paths)):
#     svar[i].attrs = ''
#     svar[i].name = 'svar'
#     p = 'svar_vavg_whole_u3hc4_'+'ensemble_'+str(i)+'.nc'
#     svar[i].to_netcdf(p, mode = 'w')
    
# for i in range(len(paths)):
#     svar_1m[i].attrs = ''
#     svar_1m[i].name = 'svar'
#     p = 'svar_vavg_top1m_u3hc4_'+'ensemble_'+str(i)+'.nc'
#     svar_1m[i].to_netcdf(p, mode = 'w')

# for i in range(len(paths)):
#     p = 'ene_vint_whole_u3hc4_'+'ensemble_'+str(i)+'.nc'
#     dse[i].to_netcdf(p, mode = 'w')
    
# for i in range(len(paths)):
#     p = 'mix_vint_whole_u3hc4_'+'ensemble_'+str(i)+'.nc'
#     dsm[i].to_netcdf(p, mode = 'w')
    
# for i in range(len(paths)):
#     p = 'mix_vint_top1m_u3hc4_'+'ensemble_'+str(i)+'.nc'
#     dsm_1m[i].to_netcdf(p, mode = 'w')